## Saving Samples [mchirp,q,chieff,z] from Normalising flow models to HDF5 file

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch
import time
from scipy.special import logsumexp

import sys

sys.path.append('../')
from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

from sample import sample

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


glasflow is using its own internal version of nflows


In [2]:
params = ['mchirp','q', 'chieff', 'z']
channels = ['CE','CHE','GC','NSC','SMT']
chi_b = [0.0,0.1,0.2,0.5]
alpha = [0.2,0.5,1.,2.,5.]
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'

model_names, KDEs = read_models.get_models(file_path, channels, params, use_flows=False, device='cpu')
model_names.sort()
hyperparams = sorted(list(set([x.split('/', 1)[1] for x in model_names])))
Nhyper = np.max([len(x.split('/')) for x in hyperparams])

100%|██████████| 36/36 [00:18<00:00,  1.90it/s]


In [3]:
hyperparam_dict  = {}
hyperidx=0
while hyperidx < Nhyper:
    hyperidx_with_Nhyper = np.argwhere(np.asarray([len(x.split('/')) for x in hyperparams])>hyperidx).flatten()
    hyperparams_at_level = sorted(set([x.split('/')[hyperidx] for x in np.asarray(hyperparams)[hyperidx_with_Nhyper]]))
    hyperparam_dict[hyperidx] = hyperparams_at_level
    hyperidx += 1
all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])
while all_models_at_deepest==False:
    # loop until all models have the same length
    for model in model_names:
        # See number of hyperparameters in model, subtract one for channel
        Nhyper_in_model = len(model.split('/'))-1
        # loop until this model has all the hyperparam levels as well
        while Nhyper_in_model < Nhyper:
            model_names.remove(model)
            for new_hyperparam in hyperparam_dict[Nhyper_in_model]:
                # add new model name
                model_names.append(model+'/'+new_hyperparam)
            Nhyper_in_model += 1
        model_names.sort()
    # see if all models are at deepest level else repeat
    all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])

In [4]:
#inputs: x, data, pop_models, submodels_dict, channels, use_flows
hyperparams = list(set([x.split('/', 1)[1] for x in model_names]))
Nhyper = np.max([len(x.split('/')) for x in hyperparams])
channels = sorted(list(set([x.split('/')[0] for x in model_names])))

# construct dict that relates submodels to their index number
submodels_dict = {} #dummy index dict keys:0,1,2,3, items: particular models
ctr=0 #associates with either chi_b or alpha (0 or 1)
while ctr < Nhyper:
    submodels_dict[ctr] = {}
    hyper_set = sorted(list(set([x.split('/')[ctr] for x in hyperparams])))
    for idx, model in enumerate(hyper_set): #idx associates with 0,1,2,3,(4) keys
        submodels_dict[ctr][idx] = model
    ctr += 1

In [5]:
submodels_dict

{0: {0: 'chi00', 1: 'chi01', 2: 'chi02', 3: 'chi05'},
 1: {0: 'alpha02', 1: 'alpha05', 2: 'alpha10', 3: 'alpha20', 4: 'alpha50'}}

In [6]:
all_models = []
models = []
def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])

In [7]:
f = h5py.File(file_path, 'r')
f.visititems(find_submodels)
# get all unique models
all_models = sorted(list(set(all_models)))
f.close()

In [16]:
submodels_dict[1][1]

'alpha05'

In [18]:
for chnl in channels:
    for chibid in range(4):
        for alphaid in range(5):
            if chnl == 'CE':
                samples = KDEs[chnl][submodels_dict[0][chibid]][submodels_dict[1][chibid]].sample(1000000)
                df = pd.DataFrame(data=samples, columns=params)
                df.to_hdf("KDE_samples.hdf5", key=f'{chnl}/{submodels_dict[0][chibid]}/{submodels_dict[1][alphaid]}')
            else:
                samples = KDEs[chnl][submodels_dict[0][chibid]].sample(1000000)
                df = pd.DataFrame(data=samples, columns=params)
                df.to_hdf("KDE_samples.hdf5", key=f'{chnl}/{submodels_dict[0][chibid]}')